# Data Integration

## From Web Scraping data

In [35]:
import pandas as pd

# Read CSV files into DataFrames
bosses_df = pd.read_csv("data/bosses_df.csv")
enemies_df = pd.read_csv("data/enemies_df.csv")
characters_df = pd.read_csv("data/characters_df.csv")
games_df = pd.read_csv("data/games_data.csv")
sales_df = pd.read_csv("data/sales_data.csv")

In [36]:
# Merge the DataFrames, renaming columns to 'Figure' where appropriate
merged_data = pd.concat([
    bosses_df[['Game', 'Relation', 'Boss', 'Species']].rename(columns={'Boss': 'Figure'}),
    characters_df[['Game', 'Relation', 'Character', 'Species']].rename(columns={'Character': 'Figure'}),
    enemies_df[['Game', 'Relation', 'Enemy']].rename(columns={'Enemy': 'Figure'})
], ignore_index=True)

merged_data

,Game,Relation,Figure,Species
0,Mario & Luigi: Dream Team,BOSS_IN,Antasma,Bat
1,Mario & Luigi: Paper Jam,BOSS_IN,Antasma,Bat
2,Mario Kart DS,BOSS_IN,Black Shy Guy,NaN
3,Paper Mario,BOSS_IN,Black Shy Guy,NaN
4,Mario Kart 8 / Mario Kart 8 Deluxe,BOSS_IN,Black Shy Guy,NaN
...,...,...,...,...
7307,Super Mario Bros. Wonder,ENEMY_IN,Wiggler,NaN
7308,Super Mario Party Jamboree,ENEMY_IN,Wiggler,NaN
7309,Super Smash Bros. Brawl,ENEMY_IN,Zinger,NaN
7310,Super Smash Bros. for Wii U,ENEMY_IN,Zinger,NaN


In [37]:
# Add 'Console' and 'Year' to merged_data
merged_data = merged_data.merge(
    games_df[['Game', 'Console', 'Year']],
    on='Game',
    how='left'
).assign(Year=lambda df: pd.to_numeric(df['Year'], errors='coerce').astype('Int64'))


# Remove underscores and quotes, and also parentheses from 'Figure'
merged_data['Figure'] = merged_data['Figure'].str.replace(r'[_"]', ' ', regex=True)
merged_data['Figure'] = merged_data['Figure'].str.replace(r'\s*\(.*\)', '', regex=True).str.strip()


# Remove rows with NaN values in 'Year' and 'Console'
merged_data = merged_data.dropna(subset=['Year', 'Console'])

# Convert specified columns to lowercase, correctly handling missing values
for column in ['Game', 'Console', 'Figure', 'Species']:
    if column in merged_data.columns:
        # Select only non-null values
        not_null_mask = merged_data[column].notna()
        # Apply lowercase only to the non-null values
        merged_data.loc[not_null_mask, column] = merged_data.loc[not_null_mask, column].astype(str).str.lower()
    else:
        print(f"Warning: Column '{column}' not found in the dataset.")

In [38]:
# Display the dataframe
merged_data

,Game,Relation,Figure,Species,Console,Year
0,mario & luigi: dream team,BOSS_IN,antasma,bat,nintendo 3ds,2013
1,mario & luigi: paper jam,BOSS_IN,antasma,bat,nintendo 3ds,2015
2,mario kart ds,BOSS_IN,black shy guy,NaN,nintendo ds,2005
3,paper mario,BOSS_IN,black shy guy,NaN,nintendo 64,2000
5,paper mario: color splash,BOSS_IN,black shy guy,NaN,wii u,2016
...,...,...,...,...,...,...
8630,super mario bros. wonder,ENEMY_IN,wiggler,NaN,nintendo switch,2023
8631,super mario party jamboree,ENEMY_IN,wiggler,NaN,nintendo switch,2024
8632,super smash bros. brawl,ENEMY_IN,zinger,NaN,wii,2008
8633,super smash bros. for wii u,ENEMY_IN,zinger,NaN,wii u,2014


In [39]:
# Dictionary to map original values to new values, which are the primary console names

console_mapping = {
    'nes': 'NES',
    'famicom / nes': 'NES',
    'famicom': 'NES',
    'nes (disk system)': 'NES',
    'snes': 'SNES',
    'super famicom/snes': 'SNES',
    'super nintendo': 'SNES',
    'super famicom / snes': 'SNES',
    'super famicom': 'SNES',
    'satellaview (super famicom)': 'SNES',
    'rom cartridge / satellaview (super famicom)': 'SNES',
    'super famicom (np)': 'SNES',
    'n64': 'N64',
    'nintendo 64': 'N64',
    'nintendo 64 (64dd)': 'N64',
    'gc': 'GC',
    'gcn': 'GC',
    'nintendo gamecube': 'GC',
    'wii': 'Wii',
    'wii (wiiware)': 'Wii',
    'wiiware': 'Wii',
    'wiiu': 'WiiU',
    'wii u': 'WiiU',
    'switch': 'Switch',
    'nintendo switch': 'Switch',
    'switch 2': 'Switch 2',
    'nintendo switch 2': 'Switch 2',
    'nintendo switch (oled)': 'Switch',
    'nintendo entertainment system': 'NES',
    'super nintendo entertainment system': 'SNES',

    # Nintendo
    'gb': 'GB',
    'game boy': 'GB',
    'gamecube': 'GC',
    'gbc': 'GBC',
    'game boy color': 'GBC',
    'game boy/game boy color': 'GBC',
    'gba': 'GBA',
    'game boy advance': 'GBA',
    'game boy advance (e-reader)': 'GBA',
    'ds': 'DS',
    'nintendo ds': 'DS',
    'nintendo dsi (dsiware)': 'DS',
    'nintendo dsi': 'DS',
    '3ds': '3DS',
    'nintendo 3ds': '3DS',
    'new nintendo 3ds': '3DS',
    'dsiware': 'DS',
    'nintendo 3ds (eshop)': '3DS',
    'nintendo 64dd': 'N64',

    # Arcade
    'arcade': 'Arcade',
    'vs. system': 'Arcade',
    'arcade edition - arcade': 'Arcade',
    'triforce arcade': 'Arcade',
    'arcade, nintendo entertainment system': 'Arcade',

    # PC
    'pc': 'PC',
    'personal computer': 'PC',
    'family computer disk system': 'PC',
    'family computer': 'PC',
    'ms-dos': 'PC',
    'windows': 'PC',
    'microsoft windows': 'PC',
    'mac os': 'PC',
    'microsoft windows/macintosh': 'PC',
    'apple ii': 'PC',
    'commodore 64': 'PC',
    'commodore vic-20': 'PC',
    'commodore 64 (ocean software)': 'PC',
    'amstrad cpc': 'PC',
    'zx spectrum': 'PC',
    'msx': 'PC',
    'pc-8001': 'PC',
    'pc-88': 'PC',
    'nec pc-8801': 'PC',
    'nec pc-6001mkii/6601, nec pc-8001, nec pc-8801mkii, nec pc-9801, fm-7, sharp x1, sharp mz-1500, sharp mz-2200, hitachi s1, smc-777': 'PC',
    'nec pc-6001mkii/6601, nec pc-8001mkii,nec pc-8801, fm-7, sharp x1, sharp mz-1500, hitachi s1, ibm jx': 'PC',
    'nec pc-6001mkii/6601, nec pc-8801, sharp x1, ibm jx': 'PC',
    'sharp x1': 'PC',

    # Mobile
    'mobile': 'Mobile',
    'ios': 'Mobile',
    'android': 'Mobile',
    'ios/android': 'Mobile',
    'ios, android': 'Mobile',
    'ios, ipados, android': 'Mobile',
    'ios / android': 'Mobile',

    # Atari Consoles
    '2600': '2600',
    'atari 2600': '2600',
    'atari 5200': 'Atari 5200',
    'atari 7800': 'Atari 7800',
    'atari 8-bit computers': 'Atari 8-bit',
    'atari 8-bit': 'Atari 8-bit',

    # Other Consoles
    'colecovision': 'ColecoVision',
    'intellivision': 'Intellivision',
    'philips cd-i': 'Philips CD-i',
    'cd-i': 'Philips CD-i',
    'virtual boy': 'Virtual Boy',
    'ps2': 'PlayStation 2',
    'playstation 2': 'PlayStation 2',
    'psp': 'PlayStation Portable',
    'playstation portable': 'PlayStation Portable',

    # Game & Watch and similar
    'game & watch': 'Game & Watch',
    'nelsonic game watch': 'Game & Watch',
    'coleco tabletop': 'Other Handheld',
    'super mario bros. watch': 'Other Handheld',
    'gamewatch boy': 'Other Handheld',

    # Multi-platform strings (map to primary or a chosen standard)
    'famicom / nes / game boy': 'NES',
    'nes / game boy': 'NES',
    'wii u / wii / nintendo 3ds': 'WiiU',
    'wii u / nintendo 3ds': 'WiiU',
    'nintendo switch / nintendo 3ds': 'Switch',

    # Miscellaneous / Other
    'ti-99/4a': 'Other Computer',
    'coleco adam': 'Other Computer',
    'sky italia': 'Other Platform',
    'satellaview': 'Other Platform',
    'Virtual Boy': 'Other Platform',
    'e-reader': 'Other Platform',
    'super famicom (satellaview)': 'Other Platform'
}


# Apply the mapping to the 'Console' column and make the matching case-insensitive
# If there are values in the 'Console' column that are not present in the dictionary, they will remain unchanged and we will remove duplicates
merged_data.loc[:, 'Console'] = merged_data['Console'].str.lower().map(console_mapping).fillna(merged_data['Console']) 

# Columns that identify a duplicate entry
key_cols = ['Game', 'Relation', 'Figure', 'Console', 'Year']

# Within each group of duplicates, fill the 'Species' value in order to ensure that the species information is preserved.
merged_data['Species'] = merged_data.groupby(key_cols)['Species'].transform('first')

# Remove duplicates based on the key columns
merged_data = merged_data.drop_duplicates(subset=key_cols)

In [40]:
merged_data

,Game,Relation,Figure,Species,Console,Year
0,mario & luigi: dream team,BOSS_IN,antasma,bat,3DS,2013
1,mario & luigi: paper jam,BOSS_IN,antasma,bat,3DS,2015
2,mario kart ds,BOSS_IN,black shy guy,None,DS,2005
3,paper mario,BOSS_IN,black shy guy,None,N64,2000
5,paper mario: color splash,BOSS_IN,black shy guy,None,WiiU,2016
...,...,...,...,...,...,...
8630,super mario bros. wonder,ENEMY_IN,wiggler,None,Switch,2023
8631,super mario party jamboree,ENEMY_IN,wiggler,None,Switch,2024
8632,super smash bros. brawl,ENEMY_IN,zinger,None,Wii,2008
8633,super smash bros. for wii u,ENEMY_IN,zinger,None,WiiU,2014


In [42]:
# Remove rows where the 'Console' value appears less than 10 times
console_counts = merged_data['Console'].value_counts()
merged_data = merged_data[merged_data['Console'].isin(console_counts[console_counts > 10].index)]

In [43]:
# Convert specified columns to lowercase, sales_df
for column in ['Game']:
    if column in sales_df.columns:
        sales_df[column] = sales_df[column].astype(str).str.lower()
    else:
        print(f"Warning: Column '{column}' not found in the dataset.")

In [44]:
# Merge 'sales_df' with 'merged_data' on 'Game', keeping only the columns from merged_data
sales_df = sales_df.rename(columns={'Platform': 'Console'})
merged_data = merged_data.merge(
     sales_df.groupby('Game')['Sales'].sum(),
     on='Game',
     how='left')

In [46]:
# Save the final DataFrame to a CSV file
merged_data.to_csv("data/merged_data.csv", index=False)
print("✅ Merged data saved to 'merged_data.csv'.")
# Display the final merged DataFrame
merged_data

✅ Merged data saved to 'merged_data.csv'.


,Game,Relation,Figure,Species,Console,Year,Sales
0,mario & luigi: dream team,BOSS_IN,antasma,bat,3DS,2013,2.08
1,mario & luigi: paper jam,BOSS_IN,antasma,bat,3DS,2015,0.76
2,mario kart ds,BOSS_IN,black shy guy,None,DS,2005,23.21
3,paper mario,BOSS_IN,black shy guy,None,N64,2000,1.38
4,paper mario: color splash,BOSS_IN,black shy guy,None,WiiU,2016,0.42
...,...,...,...,...,...,...,...
6953,super mario bros. wonder,ENEMY_IN,wiggler,None,Switch,2023,NaN
6954,super mario party jamboree,ENEMY_IN,wiggler,None,Switch,2024,NaN
6955,super smash bros. brawl,ENEMY_IN,zinger,None,Wii,2008,12.84
6956,super smash bros. for wii u,ENEMY_IN,zinger,None,WiiU,2014,NaN


## From API data

In [30]:
# Prepare API Relationship data

# Create a master map of characters to their species from the initial data.
# This will be used to enrich the relationship data.
initial_figures_df = pd.concat([
    bosses_df[['Boss', 'Species']].rename(columns={'Boss': 'Figure'}),
    characters_df[['Character', 'Species']].rename(columns={'Character': 'Figure'}),
    enemies_df[['Enemy']].rename(columns={'Enemy': 'Figure'})
], ignore_index=True)
initial_figures_df['Figure'] = initial_figures_df['Figure'].str.replace(r'[_"]', ' ', regex=True)
initial_figures_df['Figure'] = initial_figures_df['Figure'].str.lower().str.replace(r'\s*\(.*\)', '', regex=True).str.strip()
character_species_map = initial_figures_df.dropna(subset=['Figure', 'Species']).drop_duplicates(subset=['Figure']).set_index('Figure')['Species']
#character_species_map = initial_figures_df.dropna(subset=['Figure']).drop_duplicates(subset=['Figure']).set_index('Figure')['Species']

# Load and process the relationships data from the API
relationships_df = pd.read_csv("data/general_character_relationships.csv")
relationships_df['RelationshipType'] = relationships_df['RelationshipType'].replace({
    'Friend': 'FRIEND_WITH',
    'Enemy': 'ENEMY_WITH'
})
# Ensure 'Character' and 'RelatedCharacter' are in lowercase
relationships_df['Character'] = relationships_df['Character'].str.lower()
relationships_df['RelatedCharacter'] = relationships_df['RelatedCharacter'].str.lower()

# Enrich the relationship data with species information using the map
relationships_df['FigureSpecies'] = relationships_df['Character'].map(character_species_map)
relationships_df['RelatedCharacterSpecies'] = relationships_df['RelatedCharacter'].map(character_species_map)

In [31]:
# Create the final API relationship file

# Rename 'Character' to 'Figure' for consistency with the graph importer
merged_data_API = relationships_df.rename(columns={'Character': 'Figure'})

# Clean up and save the final API data file
merged_data_API.dropna(subset=['Figure', 'RelatedCharacter', 'RelationshipType'], inplace=True)
merged_data_API = merged_data_API.drop_duplicates().reset_index(drop=True)
merged_data_API.to_csv("data/merged_data_API.csv", index=False)

print("✅ Enriched API relationship data saved to 'merged_data_API.csv'.")
merged_data_API

✅ Enriched API relationship data saved to 'merged_data_API.csv'.


,Figure,RelatedCharacter,RelationshipType,FigureSpecies,RelatedCharacterSpecies
0,mario,admiral bobbery,FRIEND_WITH,Human,NaN
1,mario,birdo,FRIEND_WITH,Human,NaN
2,mario,bombette,FRIEND_WITH,Human,NaN
3,mario,bowser,FRIEND_WITH,Human,Koopa
4,mario,brighton,FRIEND_WITH,Human,NaN
...,...,...,...,...,...
2519,zoda,luna ryder,ENEMY_WITH,NaN,NaN
2520,zoda,mrs. arrow,ENEMY_WITH,NaN,NaN
2521,zoda,rick wheeler,ENEMY_WITH,NaN,NaN
2522,zoda,samurai goroh,ENEMY_WITH,NaN,NaN
